In [2]:
import numpy as np
import keras
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.WARN)  # or any {DEBUG, INFO, WARN, ERROR, FATAL}

In this exercise I am going to rewrite a dense layer, tf.layers.dense as a convolutional layer, tf.layers.conv2d. The underlying math will be the same, but the spatial information will be preserved allowing seamless use of future convolutional layers.

In [3]:
# custom init with the seed set to 0 by default
def custom_init(shape,dtype=tf.int32,partition_info=None,seed=0):
    
    return tf.random_normal(shape,dtype=dtype,seed=seed)


In [4]:
#Use `tf.layers.conv2d` to reproduce the result of `tf.layers.dense`.
# Set the `kernel_size` and `stride`.
def conv_1x1(x,num_output):
    kernel_size=1
    stride=1
    return tf.layers.conv2d(x,num_output,kernel_size,stride,kernel_initializer=custom_init)


In [5]:
num_outputs=6
x=tf.constant(np.random.randn(1,2,2,1),dtype=tf.float32)

One way to think about one by one convolutional layers is the number of kernels is equivalent to the number of outputs in a fully connected layer. Similarly, the number of weights in each kernel is equivalent to the number of inputs in the fully connected layer. Effectively, this turns convolutions into a matrix multiplication with spatial information. 

In [7]:
dense_out = tf.layers.dense(x, num_outputs,kernel_initializer=custom_init)
conv_out = conv_1x1(x, num_outputs)


In [8]:
print(dense_out.shape)
print(conv_out.shape)

(1, 2, 2, 6)
(1, 2, 2, 6)


In [9]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    a = sess.run(dense_out)
    b = sess.run(conv_out)
    print("Dense Output =", a)
    print("Conv 1x1 Output =", b)

    print("Same output? =", np.allclose(a, b, atol=1.e-5))

Dense Output = [[[[ 0.50380087 -2.6561394  -0.21592058 -0.68979347  3.0587225
    -0.5333308 ]
   [ 0.3227102  -1.7013931  -0.13830817 -0.44184795  1.9592681
    -0.34162566]]

  [[ 0.07980407 -0.4207431  -0.03420268 -0.10926604  0.48451388
    -0.08448173]
   [ 0.27763024 -1.4637225  -0.11898765 -0.38012543  1.6855744
    -0.29390335]]]]
Conv 1x1 Output = [[[[ 0.50380087 -2.6561394  -0.21592058 -0.68979347  3.0587225
    -0.5333308 ]
   [ 0.3227102  -1.7013931  -0.13830817 -0.44184795  1.9592681
    -0.34162566]]

  [[ 0.07980407 -0.4207431  -0.03420268 -0.10926604  0.48451388
    -0.08448173]
   [ 0.27763024 -1.4637225  -0.11898765 -0.38012543  1.6855744
    -0.29390335]]]]
Same output? = True


The correct use is tf.layers.conv2d(x, num_outputs, 1, 1, kernel_initializer=custom_init).

    num_outputs defines the number of output channels or kernels
    The third argument is the kernel size, which is 1.
    The fourth argument is the stride, we set this to 1.
    We use the custom initializer so the weights in the dense and convolutional layers are identical.

This results in the a matrix multiplication operation that preserves spatial information.
